In [1]:
import torch
from transformers import get_constant_schedule_with_warmup
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import os
from dataloader import CoNLLReader
from tqdm import tqdm
from tutils import *

In [2]:
try:
    del globals()['NERmodelbase']
except:
    pass
from NERmodel3 import NERmodelbase3

In [3]:
encoder_model = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(encoder_model)

In [4]:
def collate_batch(batch):
    batch_ = list(zip(*batch))
    tokens, masks, token_masks, gold_spans, tags, lstm_encoded = batch_[0], batch_[1], batch_[2], batch_[3], \
                                                                 batch_[4], batch_[5]
    # print(tags)
    max_len = np.max([len(token) for token in tokens])
    # print(np.max([len(token) for token in tokens]), max_len)
    token_tensor = torch.empty(size=(len(tokens), max_len), dtype=torch.long).fill_(tokenizer.pad_token_id)
    tag_tensor = torch.empty(size=(len(tokens), max_len), dtype=torch.long).fill_(mconern['O'])
    mask_tensor = torch.zeros(size=(len(tokens), max_len), dtype=torch.bool)
    token_masks_tensor = torch.zeros(size=(len(tokens), max_len), dtype=torch.bool)
    lstm_encoded_tensor = torch.zeros(size=(len(tokens), max_len, 256), dtype=torch.float)
    # print(lstm_encoded.shape)
    for i in range(len(tokens)):
        tokens_ = tokens[i]
        seq_len = len(tokens_)

        token_tensor[i, :seq_len] = tokens_
        tag_tensor[i, :seq_len] = tags[i]
        mask_tensor[i, :seq_len] = masks[i]
        token_masks_tensor[i, :seq_len] = token_masks[i]
        lstm_encoded_tensor[i, 1:seq_len - 1, :] = lstm_encoded[i]

    return token_tensor, tag_tensor, mask_tensor, token_masks_tensor, gold_spans, lstm_encoded_tensor

In [5]:
def get_optimizer(net, opt=False):
    optimizer = torch.optim.AdamW(net.parameters(), lr=1e-4, weight_decay=0.03)
    if opt:
        scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEP)
        return [optimizer], [scheduler]
    return [optimizer]


In [6]:
NUM_EPOCH = 1
BATCH_SIZE = 64
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [7]:
# wnut_iob = {'B-CORP': 0, 'I-CORP': 1, 'B-CW': 2, 'I-CW': 3, 'B-GRP': 4, 'I-GRP': 5, 'B-LOC': 6, 'I-LOC': 7,
#             'B-PER': 8, 'I-PER': 9, 'B-PROD': 10, 'I-PROD': 11, 'O': 12}
mconern = indvidual(mconer_grouped, True)
reveremap = invert(mconer_grouped)

In [8]:
import pickle
if os.path.exists('train_load.pkl'):
    with open('train_load.pkl', 'rb') as f:
        ds = pickle.load(f)
else:
    print("reading from disk")
    ds = CoNLLReader(target_vocab=mconern, encoder_model=encoder_model, reversemap=reveremap, finegrained=fine)
    ds.read_data(data=r'C:\Users\Rah12937\PycharmProjects\mconer\multiconer2023\train_dev\en-train.conll')
    with open('train_load.pkl', 'wb') as f:
        pickle.dump(ds, f)

In [9]:
if os.path.exists('valid_load.pkl'):
    with open('valid_load.pkl', 'rb') as f:
        valid = pickle.load(f)
else:
    valid = CoNLLReader(target_vocab=mconern, encoder_model=encoder_model, reversemap=reveremap, finegrained=True)
    valid.read_data(data=r'C:\Users\Rah12937\PycharmProjects\mconer\multiconer2023\train_dev\en-dev.conll')
    with open('valid_load.pkl', 'wb') as f:
        pickle.dump(ds, f)

In [10]:
model = NERmodelbase3(tag_to_id=mconern, device=device, encoder_model=encoder_model, dropout=0.3, use_lstm=True).to(
    device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\Rah12937\PycharmProjects\mconer\NERmodel3.py:27: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.w_omega, -0.1, 0.1)


In [11]:
trainloader = DataLoader(ds, batch_size=BATCH_SIZE, collate_fn=collate_batch, num_workers=0, shuffle=False)
validloader = DataLoader(valid, batch_size=BATCH_SIZE, collate_fn=collate_batch, num_workers=0)

In [12]:
WARMUP_STEP = int(len(trainloader) * NUM_EPOCH * 0.1)
print(f"Number of warm up step is {WARMUP_STEP}")
optim, scheduler = get_optimizer(model, True)

Number of warm up step is 26


In [13]:
from pytorchtools import EarlyStopping
import random
from tensorboardX import SummaryWriter
run_id = random.randint(1, 10000)
run_name = f"runid_{run_id}_EP_{NUM_EPOCH}_fine_xlm-b-birnnn-focal-loss-0_8-sep_lr-alpha-2-gama-4"
writer = SummaryWriter(run_name)
step = 0
running_loss = 0
early_stopping = EarlyStopping(patience=10, verbose=True, path=run_name + '.pt')

In [15]:
import numpy as np
eval_step = 100
for epoch in range(NUM_EPOCH):
    val_track = []
    with tqdm(trainloader, unit='batch') as tepoch:
        # model.train()
        tepoch.set_description(f"Epoch {epoch}")
        for i, data in enumerate(tepoch):
            optim[0].zero_grad()
            outputs, focal_loss, all_prob, token_scores, mask, metadata = model(data)
            loss = 0.2 * outputs['loss'] + 0.8 * focal_loss
            running_loss += loss
            loss.backward()
            optim[0].step()
            scheduler[0].step()
            # if i % 10 == 0:  # print every 2000 mini-batches
            model.spanf1.reset()
            # writer.add_scalar('lr', scheduler[0].get_last_lr()[0], step)
            # run validation
            step += 1
            if (step + 1) % eval_step == 0:
                model.eval()
                with torch.no_grad():
                    with tqdm(validloader, unit='batch') as tepoch:
                        val_loss = 0
                        for i, data in enumerate(tepoch):
                            outputs, focal_loss, all_prob, token_scores, mask, metadata = model(data, mode='predict')
                            val_loss += 0.2 * outputs['loss'] + 0.8 * focal_loss
                model.train()
                writer.add_scalars("Loss",
                                   {
                                       "Train Loss": round(running_loss.detach().cpu().numpy().ravel()[0] / 20, 4),
                                       "Valid Loss": round(val_loss.detach().cpu().numpy().ravel()[0] / 20, 4),
                                   }
                                   , step)
                writer.add_scalars("Metrics", outputs['results'], step)
                # print(outputs['results'])
                # writer.add_scalar("Loss/Test", round(val_loss.numpy()[0] / len(validloader), 4), step)
                # writer.add_scalar("Loss/Valid",
                #                   round(val_loss.detach().cpu().numpy().ravel()[0] / 20, 4), step)
                val_track.append(round(val_loss.detach().cpu().numpy().ravel()[0] / eval_step, 4))
                running_loss = 0

                early_stopping(round(val_loss.detach().cpu().numpy().ravel()[0] / eval_step, 4), model)
                if early_stopping.early_stop:
                    print("Stopping early")
                    break

writer.close()

  7%|████████████▏                                                                                                                                                              | 1/14 [00:00<00:07,  1.77batch/s]

tensor([[-1.1259e+01, -1.0008e+04, -8.2031e+00,  ..., -8.7335e+00,
         -1.0009e+04, -1.0547e+00],
        [-1.0671e+01, -1.9902e+01, -1.0335e+01,  ..., -9.6462e+00,
         -1.7437e+01, -1.0900e+00],
        [-7.3997e+00, -1.6369e+01, -7.2292e+00,  ..., -6.5972e+00,
         -1.5234e+01, -1.5071e+00],
        ...,
        [-3.7593e+01, -3.7997e+01, -3.7270e+01,  ..., -3.6562e+01,
         -3.7791e+01, -2.7648e+01],
        [-3.7563e+01, -4.6462e+01, -3.7224e+01,  ..., -3.6527e+01,
         -4.5734e+01, -2.7679e+01],
        [-3.9265e+01, -4.5919e+01, -3.8370e+01,  ..., -3.6927e+01,
         -4.8371e+01, -2.6347e+01]], device='cuda:0') tensor([72, 50, 51,  ..., 72, 72, 72], device='cuda:0')
tensor(3.5233, device='cuda:0')



 14%|████████████████████████▍                                                                                                                                                  | 2/14 [00:00<00:05,  2.16batch/s]

tensor([[-1.1323e+01, -1.0009e+04, -8.2771e+00,  ..., -8.8125e+00,
         -1.0009e+04, -1.0540e+00],
        [-1.0753e+01, -2.0031e+01, -1.0424e+01,  ..., -9.7350e+00,
         -1.7653e+01, -1.0880e+00],
        [-5.8942e+00, -1.5405e+01, -5.9268e+00,  ..., -5.4355e+00,
         -1.4151e+01, -3.3925e+00],
        ...,
        [-7.2552e+01, -7.4089e+01, -7.2614e+01,  ..., -7.2125e+01,
         -7.3353e+01, -7.1020e+01],
        [-7.5641e+01, -7.7182e+01, -7.5714e+01,  ..., -7.5236e+01,
         -7.6463e+01, -7.4142e+01],
        [-8.5666e+01, -8.3933e+01, -8.4774e+01,  ..., -8.3315e+01,
         -8.6942e+01, -7.2811e+01]], device='cuda:0') tensor([72, 40, 41,  ..., 41, 41, 72], device='cuda:0')
tensor(4.2999, device='cuda:0')



 21%|████████████████████████████████████▋                                                                                                                                      | 3/14 [00:01<00:04,  2.30batch/s]

...


 93%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊            | 13/14 [00:05<00:00,  2.49batch/s]

tensor([[-8.6756e+00, -1.0007e+04, -7.6055e+00,  ..., -8.0581e+00,
         -1.0008e+04, -1.0618e+00],
        [-8.5299e+00, -1.5823e+01, -1.0321e+01,  ..., -9.4964e+00,
         -1.6507e+01, -1.1006e+00],
        [-6.4063e+00, -1.4054e+01, -7.6257e+00,  ..., -6.9917e+00,
         -1.5492e+01, -1.3632e+00],
        ...,
        [-7.2599e+01, -7.4151e+01, -7.4107e+01,  ..., -7.3388e+01,
         -7.5276e+01, -6.6088e+01],
        [-7.2813e+01, -7.9255e+01, -7.4352e+01,  ..., -7.3631e+01,
         -8.1038e+01, -6.6150e+01],
        [-7.5218e+01, -7.9412e+01, -7.6311e+01,  ..., -7.4792e+01,
         -8.4747e+01, -6.4831e+01]], device='cuda:0') tensor([72, 72, 72,  ..., 72, 72, 72], device='cuda:0')
tensor(2.7744, device='cuda:0')



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.50batch/s]

tensor([[-8.7615e+00, -1.0007e+04, -7.7149e+00,  ..., -8.1475e+00,
         -1.0008e+04, -1.0599e+00],
        [-8.4746e+00, -1.5815e+01, -1.0276e+01,  ..., -9.4345e+00,
         -1.6516e+01, -1.1000e+00],
        [-6.4489e+00, -1.4010e+01, -7.6745e+00,  ..., -7.0442e+00,
         -1.5481e+01, -1.3455e+00],
        ...,
        [-1.8276e+01, -2.3710e+01, -1.9880e+01,  ..., -1.9127e+01,
         -2.5467e+01, -1.1359e+01],
        [-1.8243e+01, -2.5060e+01, -1.9824e+01,  ..., -1.9087e+01,
         -2.6952e+01, -1.1414e+01],
        [-2.0315e+01, -2.4681e+01, -2.1363e+01,  ..., -1.9858e+01,
         -3.0016e+01, -1.0098e+01]], device='cuda:0') tensor([72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
        72, 72, 28, 29, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 28, 29, 29,
        29, 29, 29, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
        72, 72, 28, 29, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 54,
        55, 55, 55, 72, 72, 28,


Epoch 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 263/263 [03:18<00:00,  1.32batch/s]


In [69]:
torch.cat(all_prob, dim=1).argmax(-1).shape

torch.Size([1, 238])

In [76]:
torch.argmax(torch.cat(all_prob, dim=1), dim=-1).squeeze(0)
torch.masked_select(metadata, mask)
# torch.nn.CrossEntropyLoss()(torch.cat(all_prob, dim=1).squeeze(0),
#                          torch.masked_select(metadata, mask))
from sklearn.metrics import f1_score
f1_score(torch.cat(all_prob, dim=1).argmax(-1).squeeze(0).cpu(),
        torch.masked_select(metadata, mask).cpu(), average='micro')

0.7268907563025211

In [17]:
from kornia.losses import FocalLoss
fl = FocalLoss(alpha=2, gamma=5, reduction='mean')
fl(token_scores.permute(0, 2, 1), metadata)

tensor(1.8132, device='cuda:0', grad_fn=<MeanBackward0>)

In [18]:
fl(torch.cat(all_prob, dim=1).squeeze(0),
    torch.masked_select(metadata, mask))

tensor(3.7399, device='cuda:0')

In [19]:
mask.unsqueeze(-1).expand(token_scores.size()).shape

torch.Size([10, 34, 73])

In [20]:
torch.masked_select(token_scores, mask.unsqueeze(-1).expand(token_scores.size())).ravel().shape

torch.Size([17374])

In [21]:
torch.masked_select(metadata, mask).shape

torch.Size([238])

In [22]:
all_prob[0].shape

torch.Size([1, 17, 73])

In [23]:
model.crf.transitions.cpu().detach().numpy().shape

(73, 73)

In [24]:
model.crf.viterbi_tags(torch.randn([2, 10, 73]))

([([4, 56, 10, 24, 16, 12, 0, 72, 28, 64], 24.494735717773438),
  ([4, 5, 16, 20, 22, 62, 56, 28, 18, 20], 23.203876495361328)],
 [tensor([[[-2.7083e+00, -9.9991e+03, -3.2452e+00, -1.0000e+04,  3.3397e+00,
            -9.9985e+03, -8.0218e-01, -9.9996e+03, -5.2809e-02, -9.9976e+03,
            -1.6960e+00, -1.0000e+04,  8.0595e-01, -9.9994e+03, -1.1864e+00,
            -9.9993e+03,  1.1634e+00, -9.9992e+03, -3.3863e+00, -1.0001e+04,
             2.3295e+00, -1.0000e+04,  6.1551e-01, -1.0001e+04, -3.6707e-01,
            -1.0001e+04, -9.2977e-01, -9.9997e+03,  4.8140e-01, -1.0001e+04,
            -2.1461e+00, -9.9992e+03, -2.0133e+00, -9.9994e+03, -1.0128e+00,
            -1.0001e+04, -4.6495e-02, -1.0000e+04,  2.6608e+00, -9.9999e+03,
            -4.0778e-01, -9.9998e+03, -4.0299e+00, -1.0000e+04, -1.1274e-01,
            -9.9996e+03, -9.5995e-02, -9.9989e+03,  1.9019e+00, -9.9996e+03,
            -2.8813e+00, -9.9999e+03, -2.0686e-01, -9.9990e+03,  7.8721e-01,
            -1.0000e+04,

In [25]:
import allennlp.nn.util as util
def viterbi_tags(
    logits: torch.Tensor, mask: torch.BoolTensor = None, top_k: int = None,
    _constraint_mask= None, _transitions=None, start_transitions=None, end_transitions=None
):
    if mask is None:
        mask = torch.ones(*logits.shape[:2], dtype=torch.bool, device=logits.device)

    if top_k is None:
        top_k = 1
        flatten_output = True
    else:
        flatten_output = False
    all_prob_val = []
    _, max_seq_length, num_tags = logits.size()
#     print(num_tags)
    # Get the tensors out of the variables
    logits, mask = logits.data, mask.data

    # Augment transitions matrix with start and end transitions
    start_tag = num_tags
    end_tag = num_tags + 1
    transitions = torch.full((num_tags + 2, num_tags + 2), -10000.0, device=logits.device)

    # Apply transition constraints
    constrained_transitions = _transitions * _constraint_mask[:num_tags, :num_tags] + -10000.0 * (1 - _constraint_mask[:num_tags, :num_tags])
    transitions[:num_tags, :num_tags] = constrained_transitions.data

    if True:
        transitions[
            start_tag, :num_tags
        ] = start_transitions.detach() * _constraint_mask[
            start_tag, :num_tags
        ].data + -10000.0 * (
            1 - _constraint_mask[start_tag, :num_tags].detach()
        )
        transitions[:num_tags, end_tag] = end_transitions.detach() * _constraint_mask[
            :num_tags, end_tag
        ].data + -10000.0 * (1 - _constraint_mask[:num_tags, end_tag].detach())
    else:
        transitions[start_tag, :num_tags] = -10000.0 * (
            1 - self._constraint_mask[start_tag, :num_tags].detach()
        )
        transitions[:num_tags, end_tag] = -10000.0 * (
            1 - self._constraint_mask[:num_tags, end_tag].detach()
        )

    best_paths = []
    # Pad the max sequence length by 2 to account for start_tag + end_tag.
    tag_sequence = torch.empty(max_seq_length + 2, num_tags + 2, device=logits.device)

    for prediction, prediction_mask in zip(logits, mask):
        mask_indices = prediction_mask.nonzero(as_tuple=False).squeeze()
        masked_prediction = torch.index_select(prediction, 0, mask_indices)
        sequence_length = masked_prediction.shape[0]

        # Start with everything totally unlikely
        tag_sequence.fill_(-10000.0)
        # At timestep 0 we must have the START_TAG
        tag_sequence[0, start_tag] = 0.0
        # At steps 1, ..., sequence_length we just use the incoming prediction
        tag_sequence[1 : (sequence_length + 1), :num_tags] = masked_prediction
        # And at the last timestep we must have the END_TAG
        tag_sequence[sequence_length + 1, end_tag] = 0.0
#         print(torch.argmax(tag_sequence, dim=-1))

        # We pass the tags and the transitions to `viterbi_decode`.
        viterbi_paths, viterbi_scores, all_prob = viterbi_decode(
            tag_sequence=tag_sequence[: (sequence_length + 2)],
            transition_matrix=transitions,
            top_k=top_k,
        )
        stacked_t = torch.stack(list(reversed(all_prob))[1:])
        all_prob_val.append(stacked_t.unsqueeze(0))
#         print(viterbi_paths, viterbi_scores)
        top_k_paths = []
        for viterbi_path, viterbi_score in zip(viterbi_paths, viterbi_scores):
            # Get rid of START and END sentinels and append.
            viterbi_path = viterbi_path[1:-1]
            top_k_paths.append((viterbi_path, viterbi_score.item()))
        best_paths.append(top_k_paths)

    if flatten_output:
        return [top_k_paths[0] for top_k_paths in best_paths]

    return best_paths, all_prob_val


In [26]:
from typing import Optional, List

def viterbi_decode(
    tag_sequence: torch.Tensor,
    transition_matrix: torch.Tensor,
    tag_observations: Optional[List[int]] = None,
    allowed_start_transitions: torch.Tensor = None,
    allowed_end_transitions: torch.Tensor = None,
    top_k: int = None,
):
    global all_score
    all_prob = []
#     print(tag_sequence.shape)
    if top_k is None:
        top_k = 1
        flatten_output = True
    elif top_k >= 1:
        flatten_output = False
    else:
        raise ValueError(f"top_k must be either None or an integer >=1. Instead received {top_k}")

    sequence_length, num_tags = list(tag_sequence.size())

    has_start_end_restrictions = (
        allowed_end_transitions is not None or allowed_start_transitions is not None
    )

    if has_start_end_restrictions:

        if allowed_end_transitions is None:
            allowed_end_transitions = torch.zeros(num_tags)
        if allowed_start_transitions is None:
            allowed_start_transitions = torch.zeros(num_tags)

        num_tags = num_tags + 2
        new_transition_matrix = torch.zeros(num_tags, num_tags)
        new_transition_matrix[:-2, :-2] = transition_matrix

        # Start and end transitions are fully defined, but cannot transition between each other.

        allowed_start_transitions = torch.cat(
            [allowed_start_transitions, torch.tensor([-math.inf, -math.inf])]
        )
        allowed_end_transitions = torch.cat(
            [allowed_end_transitions, torch.tensor([-math.inf, -math.inf])]
        )

        # First define how we may transition FROM the start and end tags.
        new_transition_matrix[-2, :] = allowed_start_transitions
        # We cannot transition from the end tag to any tag.
        new_transition_matrix[-1, :] = -math.inf

        new_transition_matrix[:, -1] = allowed_end_transitions
        # We cannot transition to the start tag from any tag.
        new_transition_matrix[:, -2] = -math.inf

        transition_matrix = new_transition_matrix

    if tag_observations:
        if len(tag_observations) != sequence_length:
            raise ConfigurationError(
                "Observations were provided, but they were not the same length "
                "as the sequence. Found sequence of length: {} and evidence: {}".format(
                    sequence_length, tag_observations
                )
            )
    else:
        tag_observations = [-1 for _ in range(sequence_length)]

    if has_start_end_restrictions:
        tag_observations = [num_tags - 2] + tag_observations + [num_tags - 1]
        zero_sentinel = torch.zeros(1, num_tags)
        extra_tags_sentinel = torch.ones(sequence_length, 2) * -math.inf
        tag_sequence = torch.cat([tag_sequence, extra_tags_sentinel], -1)
        tag_sequence = torch.cat([zero_sentinel, tag_sequence, zero_sentinel], 0)
        sequence_length = tag_sequence.size(0)

    path_scores = []
    path_indices = []

    if tag_observations[0] != -1:
        one_hot = torch.zeros(num_tags)
        one_hot[tag_observations[0]] = 100000.0
        path_scores.append(one_hot.unsqueeze(0))
    else:
        path_scores.append(tag_sequence[0, :].unsqueeze(0))

    # Evaluate the scores for all possible paths.
#     print("length", path_scores[0].shape)
#     all_score = []
    for timestep in range(1, sequence_length):
#         print(timestep, path_scores[timestep - 1].shape, transition_matrix.shape)
        # Add pairwise potentials to current scores.
        summed_potentials = path_scores[timestep - 1].unsqueeze(2) + transition_matrix
        summed_potentials = summed_potentials.view(-1, num_tags)
#         print(summed_potentials.shape, num_tags)
        # Best pairwise potential path score from the previous timestep.
        max_k = min(summed_potentials.size()[0], top_k)
        scores, paths = torch.topk(summed_potentials, k=max_k, dim=0)
#         print(scores, paths)
        all_score.append(summed_potentials)
#         print("best paths", paths)
        # If we have an observation for this timestep, use it
        # instead of the distribution over tags.
        observation = tag_observations[timestep]
        # Warn the user if they have passed
        # invalid/extremely unlikely evidence.
        if tag_observations[timestep - 1] != -1 and observation != -1:
            if transition_matrix[tag_observations[timestep - 1], observation] < -10000:
                logger.warning(
                    "The pairwise potential between tags you have passed as "
                    "observations is extremely unlikely. Double check your evidence "
                    "or transition potentials!"
                )
        if observation != -1:
            one_hot = torch.zeros(num_tags)
            one_hot[observation] = 100000.0
            path_scores.append(one_hot.unsqueeze(0))
        else:
            path_scores.append(tag_sequence[timestep, :] + scores)
        path_indices.append(paths.squeeze())
#     print("Simple Argmax", torch.argmax(torch.cat(path_scores[1:-1], dim=0), dim=-1))
#     print(torch.argmax(torch.cat(all_score, dim=0), dim=-1))
    # Construct the most likely sequence backwards.
    path_scores_v = path_scores[-1].view(-1)
#     print(path_scores[-1].shape)
    max_k = min(path_scores_v.size()[0], top_k)
    viterbi_scores, best_paths = torch.topk(path_scores_v, k=max_k, dim=0)
#     print(viterbi_scores, best_paths, max_k)
    viterbi_paths = []
    
    for i in range(max_k):
        viterbi_path = [best_paths[i]]
        
        for idx, backward_timestep in enumerate(reversed(path_indices)):
#             print(idx)
#             print("All Collected", backward_timestep, viterbi_path[-1])
#             print("recreated", all_score[-(idx+1)])
#             print("Tensor", all_score[-(idx+1)][:, viterbi_path[-1]])
            all_prob.append(all_score[-(idx+1)][:-2][:, viterbi_path[-1]])
            viterbi_path.append(int(backward_timestep.view(-1)[viterbi_path[-1]]))
        # Reverse the backward path.
        viterbi_path.reverse()

        if has_start_end_restrictions:
            viterbi_path = viterbi_path[1:-1]

        # Viterbi paths uses (num_tags * n_permutations) nodes; therefore, we need to modulo.
        viterbi_path = [j % num_tags for j in viterbi_path]
        viterbi_paths.append(viterbi_path)

    if flatten_output:
        return viterbi_paths[0], viterbi_scores[0]

    return viterbi_paths, viterbi_scores, all_prob


In [27]:
dimen = 73
z = torch.randn([2, 100, dimen])
all_prob_val = []
x, all_prob = viterbi_tags(logits=z, top_k=1,
             _constraint_mask= model.crf._constraint_mask[:dimen+2, :dimen+2], _transitions=model.crf.transitions[:dimen, :dimen],
             start_transitions=model.crf.start_transitions[:dimen],
            end_transitions = model.crf.end_transitions[:dimen])
print(torch.argmax(torch.cat(all_prob, dim=0), dim=-1))
print(x[0])
# for i in torch.tensor_split(z, len(z), dim=0):
#     all_score = []

#     final_val = []
#     x, all_prob = viterbi_tags(logits=i, top_k=1,
#                  _constraint_mask= model.crf._constraint_mask[:dimen+2, :dimen+2], _transitions=model.crf.transitions[:dimen, :dimen],
#                  start_transitions=model.crf.start_transitions[:dimen],
#                 end_transitions = model.crf.end_transitions[:dimen])
# #     print(x)
#     stacked_t = torch.stack(list(reversed(all_prob))[1:])
#     all_prob_val.append(stacked_t.unsqueeze(0))
#     print(torch.argmax(stacked_t, dim=1))
#     print(all(torch.tensor(x[0][0][0]) == torch.argmax(stacked_t, dim=1)))

NameError: name 'all_score' is not defined

In [ ]:
for i in x:
    print(i)

In [ ]:
torch.argmax(torch.stack(list(reversed(all_prob))), dim=0)


In [ ]:
print(torch.topk(all_score[-2], 1, 0))
print(all_score[-1])
all_score[-1][:, 6]

In [ ]:
p = torch.randn([1,10]).unsqueeze(2)
q = torch.randn([10,10])
# print(p,q)
model.crf._constraint_mask.shape
model.crf.transitions.shape

In [ ]:
def _viterbi_decoding(emissions, transitions):
    scores = torch.zeros(emissions.size(1))
#     print(scores)
#     back_pointers = torch.zeros(emissions.size()).int()
#     scores = scores + emissions[0]
#     # Generate most likely scores and paths for each step in sequence
#     for i in range(1, emissions.size(0)):
#         scores_with_transitions = scores.unsqueeze(1).expand_as(transitions) + transitions
#         max_scores, back_pointers[i] = torch.max(scores_with_transitions, 0)
#         scores = emissions[i] + max_scores
#     # Generate the most likely path
#     viterbi = [scores.numpy().argmax()]
#     back_pointers = back_pointers.numpy()
#     for bp in reversed(back_pointers[1:]):
#         viterbi.append(bp[viterbi[-1]])
#     viterbi.reverse()
#     viterbi_score = scores.numpy().max()
#     return viterbi_score, viterbi

In [ ]:
_viterbi_decoding(z, model.crf.transitions)

In [ ]:
torch.topk((p+q).view(-1,10), k=1, dim=0)

In [ ]:
model.crf._constraint_mask[:73, :73] *  model.crf.transitions

In [ ]:
p+q